# Import NLTK Libraries

In [60]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
BooleanOperator=["And","Or","Not"]

# Reading Files

In [61]:
def readingfromfile():
    FilesvsLines={}
    NumOfFile=int(input("Enter Number Of Files You Will Enter : "))
    for x in range(0,NumOfFile):
        NameOfFile=input("Enter Name of file : ")
        File=open(NameOfFile+".txt",'r')
        Lines = File.read().splitlines()
        FilesvsLines[NameOfFile]=Lines
        File.close()
    return FilesvsLines

In [62]:
FilesvsLines=readingfromfile()
print(FilesvsLines)

Enter Number Of Files You Will Enter : 4
Enter Name of file : DOC1
Enter Name of file : DOC2
Enter Name of file : DOC3
Enter Name of file : DOC4
{'DOC1': ['English tutorial'], 'DOC2': ['Advance English'], 'DOC3': ['English Structure'], 'DOC4': ['Tutorial']}


# Preprocessing

## Tokenization 

In [63]:
def splitwords(Words):
    TWords=[]
    for Word in Words:
        TWords.extend(word_tokenize(Word))
    return TWords    

## Remove Stop Words Using Nltk

In [64]:
def removeStopWords(words):
    filtered_sentence = []
    for word in words:
        if word not in stop_words:
            filtered_sentence.append(word.capitalize())
    return filtered_sentence       

## Remove Stop Words For Query

In [65]:
def RemoveStopWordsforQuery(Words):
    BooleanOperators=["And","Or","Not","and","or","not"]
    for index,word in enumerate(Words):
        if word in BooleanOperators:
            Words[index]=word
        elif word not in stop_words:
            Words[index]=word
        elif word in stop_words:
            Words.remove(word)
    return Words

In [66]:
print(RemoveStopWordsforQuery(["english","Ahmed","and"]))

['english', 'Ahmed', 'and']


## Preprocessing Documents

In [67]:
def PreprocessingDocuments(FilesvsLines):
    for key ,value in FilesvsLines.items():
        FilesvsLines[key]=removeStopWords(splitwords(value))
    return FilesvsLines

In [68]:
FilesvsLines=PreprocessingDocuments(FilesvsLines)
print(FilesvsLines)

{'DOC1': ['English', 'Tutorial'], 'DOC2': ['Advance', 'English'], 'DOC3': ['English', 'Structure'], 'DOC4': ['Tutorial']}


# Term - Document Incidence matrix creation

In [69]:
def AssignTermIncidenceMatrix(FilesvsLines):
    TermWithIncidenceMatrix={}
    for value in FilesvsLines.values():
        for x in value:
            vector=[]
            for key in FilesvsLines.keys():
                if x in FilesvsLines.get(key):
                    vector.append(1)
                else:
                    vector.append(0)
                TermWithIncidenceMatrix[x]=vector  
    return TermWithIncidenceMatrix   

In [70]:
TermWithIncidenceMatrix=AssignTermIncidenceMatrix(FilesvsLines)
print(TermWithIncidenceMatrix)

{'English': [1, 1, 1, 0], 'Tutorial': [1, 0, 0, 1], 'Advance': [0, 1, 0, 0], 'Structure': [0, 0, 1, 0]}


## Get Term For IncidenceVector

In [71]:
def GetTermIncidenceVector(TermWithIncidenceMatrix,Term):
    return TermWithIncidenceMatrix.get(Term)

# Query Preprocessing

In [72]:
def QueryPreprocessing(TermWithIncidenceMatrix):
    query=input("Enter Your Query : ")
    #Capitalize each word and split it in Words
    queryterm = query.split(" ")
    newquery=RemoveStopWordsforQuery(queryterm)
    QueryTerm=[]
    for i in newquery:
        QueryTerm.append(i.capitalize())
    #Removing Words That are not Exist in The Documents
    for index,x in enumerate(QueryTerm):
        if x in TermWithIncidenceMatrix.keys():
            QueryTerm[index]=x
        elif x in  BooleanOperator:
            QueryTerm[index]=x
    return QueryTerm

In [73]:
Query=QueryPreprocessing(TermWithIncidenceMatrix)
print(Query)

Enter Your Query : english or Structure
['English', 'Or', 'Structure']


## Process Boolean Operator

In [74]:
def ProcessBooleanOperator(Operation,PreviousTermVector,NextTermVector):
    resultVector=[]
    if(Operation=="Not"):
        for index,x in enumerate(PreviousTermVector):
            if x==1:
                resultVector.append(0)
            elif x==0:
                resultVector.append(1)
    elif(Operation=="And"):
        for x in range(0,len(PreviousTermVector)):
            if(PreviousTermVector[x]==1 and NextTermVector[x]==1):
                resultVector.append(1)
            else:
                resultVector.append(0)
    elif(Operation=="Or"):
        for x in range(0,len(PreviousTermVector)):
            if(PreviousTermVector[x]==0 and NextTermVector[x]==0):
                resultVector.append(0)
            else:
                resultVector.append(1)
    return resultVector            

In [75]:
def ProcessQuery(Query,mydict):
    print(Query)
    bitWiseOperator=""
    previousTermIncidenceV =[]
    nextTermIncidenceV =[]
    resultSet=[]
    hasPreviousTerm = False
    hasNotOperation = False
    for term in Query:
        if(term not in BooleanOperator):
            if(hasNotOperation):
                if(hasPreviousTerm):
                    nextTermIncidenceV = ProcessBooleanOperator("NOT",GetTermIncidenceVector(mydict,term), nextTermIncidenceV)
                else:
                    previousTermIncidenceV = ProcessBooleanOperator("NOT",GetTermIncidenceVector(mydict,term), nextTermIncidenceV)
                hasNotOperation=False    
            elif(not hasPreviousTerm):
                print(term)
                previousTermIncidenceV=GetTermIncidenceVector(mydict,term)
                resultSet=previousTermIncidenceV
                hasPreviousTerm=True
            else:
                print(term)
                nextTermIncidenceV=GetTermIncidenceVector(mydict,term)
        elif(term=="Not"):
            hasNotOperation=True
        else:
            print(term)
            bitWiseOperator=term
        if((nextTermIncidenceV!= []) and (not hasNotOperation)):
            print(previousTermIncidenceV)
            print(nextTermIncidenceV)
            resultSet=ProcessBooleanOperator(bitWiseOperator,previousTermIncidenceV,nextTermIncidenceV)
            previousTermIncidenceV=resultSet
            hasPreviousTerm=True
            nextTermIncidenceV=None
    return resultSet        

In [76]:
dlist=ProcessQuery(Query,TermWithIncidenceMatrix)

['English', 'Or', 'Structure']
English
Or
Structure
[1, 1, 1, 0]
[0, 0, 1, 0]


In [77]:
def DetermineDocuments(ResultList,FilesvsLines):
    ListOfFiles=list(FilesvsLines.keys())
    newList=[]
    documents=[]
    for index,value in enumerate(ResultList):
        if(value ==1):
            newList.append(index)
    for x in newList:
        for y in ListOfFiles:
            if(x == ListOfFiles.index(y)):
                documents.append(y)
    return documents            

In [78]:
print(DetermineDocuments(dlist,FilesvsLines))

['DOC1', 'DOC2', 'DOC3']
